In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import keras
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = np.asarray(df[name], dtype = np.float).mean()

    if sd is None:
        sd = np.asarray(df[name], dtype = np.float).std()

    df[name] = (np.asarray(df[name], dtype = np.float) - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

In [4]:

# This file is a CSV, just no CSV extension or headers
# Download from: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
df_ = pd.read_csv("/Users/tsaiflow/Desktop/NN&Sampling/data_ddos_kdd/train_PKT_sampling@100_10%.csv")

print("Read {} rows.".format(len(df_)))
#print("Read {} rows.".format(len(df1)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df_.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)
# df1.dropna(inplace=True,axis=1)

# The CSV file has no column heads, so add them
# df.columns = [
#     'duration',
#     'forw_bytes',
#     'back_bytes',
#     'total_bytes',
    
#     'outcome'
# ]

Read 19984 rows.


In [5]:
df_.head(5)

,Unnamed: 0,duration,forw_byte,back_byte,tot_byte,outcome
0,0,0.0,0,0,0,dos
1,1,0.0,1032,0,1032,dos
2,2,0.0,520,0,520,dos
3,3,0.0,1032,0,1032,dos
4,4,0.0,1032,0,1032,dos


In [6]:
cols = ['duration', 'forw_byte', 'back_byte','tot_byte','outcome']
df = df_[cols]

In [7]:
# df.drop(df.head(1).index, inplace = True)

In [8]:
df.head(5)

,duration,forw_byte,back_byte,tot_byte,outcome
0,0.0,0,0,0,dos
1,0.0,1032,0,1032,dos
2,0.0,520,0,520,dos
3,0.0,1032,0,1032,dos
4,0.0,1032,0,1032,dos


In [9]:
# np.asarray(df['duration'], dtype = np.float).mean()
# # df[name].mean()

In [10]:
# np.asarray(df['duration'], dtype = np.float).std()

In [11]:
# Now encode the feature vector

#encode_numeric_zscore(df, 'duration')

# encode_text_dummy(df, 'protocol_type')
# encode_text_dummy(df, 'service')
# encode_text_dummy(df, 'flag')
encode_numeric_zscore(df, 'forw_byte')
encode_numeric_zscore(df, 'back_byte')
encode_numeric_zscore(df, 'tot_byte')
# encode_text_dummy(df, 'land')
# encode_numeric_zscore(df, 'wrong_fragment')
# encode_numeric_zscore(df, 'urgent')
# encode_numeric_zscore(df, 'hot')
# encode_numeric_zscore(df, 'num_failed_logins')
# encode_text_dummy(df, 'logged_in')
# encode_numeric_zscore(df, 'num_compromised')
# encode_numeric_zscore(df, 'root_shell')
# encode_numeric_zscore(df, 'su_attempted')
# encode_numeric_zscore(df, 'num_root')
# encode_numeric_zscore(df, 'num_file_creations')
# encode_numeric_zscore(df, 'num_shells')
# encode_numeric_zscore(df, 'num_access_files')
# encode_numeric_zscore(df, 'num_outbound_cmds')
# encode_text_dummy(df, 'is_host_login')
# encode_text_dummy(df, 'is_guest_login')
# encode_numeric_zscore(df, 'count')
# encode_numeric_zscore(df, 'srv_count')
# encode_numeric_zscore(df, 'serror_rate')
# encode_numeric_zscore(df, 'srv_serror_rate')
# encode_numeric_zscore(df, 'rerror_rate')
# encode_numeric_zscore(df, 'srv_rerror_rate')
# encode_numeric_zscore(df, 'same_srv_rate')
# encode_numeric_zscore(df, 'diff_srv_rate')
# encode_numeric_zscore(df, 'srv_diff_host_rate')
# encode_numeric_zscore(df, 'dst_host_count')
# encode_numeric_zscore(df, 'dst_host_srv_count')
# encode_numeric_zscore(df, 'dst_host_same_srv_rate')
# encode_numeric_zscore(df, 'dst_host_diff_srv_rate')
# encode_numeric_zscore(df, 'dst_host_same_src_port_rate')
# encode_numeric_zscore(df, 'dst_host_srv_diff_host_rate')
# encode_numeric_zscore(df, 'dst_host_serror_rate')
# encode_numeric_zscore(df, 'dst_host_srv_serror_rate')
# encode_numeric_zscore(df, 'dst_host_rerror_rate')
# encode_numeric_zscore(df, 'dst_host_srv_rerror_rate')
outcomes = encode_text_index(df, 'outcome')
num_classes = len(outcomes)

# display 5 rows

df.dropna(inplace=True,axis=1)
df[0:5]
# This is the numeric feature vector, as it goes to the neural net

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,duration,forw_byte,back_byte,tot_byte,outcome
0,0.0,-0.008464,-0.009379,-0.012440,0
1,0.0,-0.008364,-0.009379,-0.012366,0
2,0.0,-0.008413,-0.009379,-0.012402,0
3,0.0,-0.008364,-0.009379,-0.012366,0
4,0.0,-0.008364,-0.009379,-0.012366,0


In [12]:
x, y = to_xy(df,'outcome')

In [13]:
# df.head(5)

In [14]:
# Create a test/train split.  33% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42)

In [15]:
import keras.backend as K
K.clear_session()

In [16]:
# Create neural net
model = Sequential()

# Used relu for activation function
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))

# output layer 
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
# model.add(Dense(y.shape[1],activation = 'sigmoid'))

In [17]:
# Print the model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 50)                550       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 4         
Total params: 1,125
Trainable params: 1,125
Non-trainable params: 0
_________________________________________________________________


In [18]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        # TODO:  Create two empty lists, self.loss and self.val_acc
        self.losses = []
        self.accs = []
        self.val_acc = []
        self.val_loss = []
    def on_batch_end(self, batch, logs={}):
        # TODO:  This is called at the end of each batch.  
        # Add the loss in logs.get('loss') to the loss list
        loss = logs.get('loss')
        acc = logs.get('acc')
        self.losses.append(loss)
        self.accs.append(acc)
    def on_epoch_end(self, epoch, logs):
        # TODO:  This is called at the end of each epoch.  
        # Add the test accuracy in logs.get('loss') to the val_acc list
        val_acc = logs.get('val_acc')
        val_loss = logs.get('val_loss')
        self.val_acc.append(val_acc)
        self.val_loss.append(val_loss)

# Create an instance of the history callback
history_cb = LossHistory()

In [19]:
from keras import optimizers

# optimizer defined
opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [20]:
## compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')


In [21]:
# model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

In [22]:
batch_size = 200
epochs = 200
model.fit(x_train,y_train,callbacks=[history_cb],validation_data=(x_test,y_test),verbose=1,epochs=epochs,batch_size=batch_size)

Train on 13389 samples, validate on 6595 samples
Epoch 1/200
13389/13389 [==============================] - 0s - loss: 0.6454 - acc: 0.8885 - val_loss: 0.5538 - val_acc: 0.9058
Epoch 2/200
13389/13389 [==============================] - 0s - loss: 0.4246 - acc: 0.8970 - val_loss: 0.3123 - val_acc: 0.9058
Epoch 3/200
13389/13389 [==============================] - 0s - loss: 0.3064 - acc: 0.8970 - val_loss: 0.2864 - val_acc: 0.9058
Epoch 4/200
13389/13389 [==============================] - 0s - loss: 0.3033 - acc: 0.8970 - val_loss: 0.2861 - val_acc: 0.9058
Epoch 5/200
13389/13389 [==============================] - 0s - loss: 0.3021 - acc: 0.8970 - val_loss: 0.2846 - val_acc: 0.9058
Epoch 6/200
13389/13389 [==============================] - 0s - loss: 0.2999 - acc: 0.8970 - val_loss: 0.2819 - val_acc: 0.9058
Epoch 7/200
13389/13389 [==============================] - 0s - loss: 0.2973 - acc: 0.8970 - val_loss: 0.2797 - val_acc: 0.9058
Epoch 8/200
13389/13389 [==============================

13389/13389 [==============================] - 0s - loss: 0.0988 - acc: 0.9487 - val_loss: 0.0935 - val_acc: 0.9556
Epoch 65/200
13389/13389 [==============================] - 0s - loss: 0.0952 - acc: 0.9570 - val_loss: 0.0939 - val_acc: 0.9591
Epoch 66/200
13389/13389 [==============================] - 0s - loss: 0.0981 - acc: 0.9532 - val_loss: 0.0956 - val_acc: 0.9591
Epoch 67/200
13389/13389 [==============================] - 0s - loss: 0.0962 - acc: 0.9535 - val_loss: 0.0992 - val_acc: 0.9486
Epoch 68/200
13389/13389 [==============================] - 0s - loss: 0.0980 - acc: 0.9540 - val_loss: 0.1101 - val_acc: 0.9392
Epoch 69/200
13389/13389 [==============================] - 0s - loss: 0.0995 - acc: 0.9518 - val_loss: 0.1102 - val_acc: 0.9392
Epoch 70/200
13389/13389 [==============================] - 0s - loss: 0.0958 - acc: 0.9525 - val_loss: 0.0921 - val_acc: 0.9556
Epoch 71/200
13389/13389 [==============================] - 0s - loss: 0.0999 - acc: 0.9491 - val_loss: 0.0911

13389/13389 [==============================] - 0s - loss: 0.0970 - acc: 0.9499 - val_loss: 0.0964 - val_acc: 0.9491
Epoch 128/200
13389/13389 [==============================] - 0s - loss: 0.0971 - acc: 0.9498 - val_loss: 0.0880 - val_acc: 0.9562
Epoch 129/200
13389/13389 [==============================] - 0s - loss: 0.0855 - acc: 0.9604 - val_loss: 0.0843 - val_acc: 0.9591
Epoch 130/200
13389/13389 [==============================] - 0s - loss: 0.0859 - acc: 0.9603 - val_loss: 0.0853 - val_acc: 0.9597
Epoch 131/200
13389/13389 [==============================] - 0s - loss: 0.1009 - acc: 0.9475 - val_loss: 0.1004 - val_acc: 0.9436
Epoch 132/200
13389/13389 [==============================] - 0s - loss: 0.0901 - acc: 0.9555 - val_loss: 0.0847 - val_acc: 0.9597
Epoch 133/200
13389/13389 [==============================] - 0s - loss: 0.0847 - acc: 0.9609 - val_loss: 0.0923 - val_acc: 0.9494
Epoch 134/200
13389/13389 [==============================] - 0s - loss: 0.0890 - acc: 0.9555 - val_loss:

13389/13389 [==============================] - 0s - loss: 0.0844 - acc: 0.9604 - val_loss: 0.0953 - val_acc: 0.9058
Epoch 191/200
13389/13389 [==============================] - 0s - loss: 0.1030 - acc: 0.9438 - val_loss: 0.1078 - val_acc: 0.9390
Epoch 192/200
13389/13389 [==============================] - 0s - loss: 0.0908 - acc: 0.9562 - val_loss: 0.0826 - val_acc: 0.9597
Epoch 193/200
13389/13389 [==============================] - 0s - loss: 0.0844 - acc: 0.9609 - val_loss: 0.0966 - val_acc: 0.9491
Epoch 194/200
13389/13389 [==============================] - 0s - loss: 0.1245 - acc: 0.9257 - val_loss: 0.1149 - val_acc: 0.9304
Epoch 195/200
13389/13389 [==============================] - 0s - loss: 0.0989 - acc: 0.9541 - val_loss: 0.0883 - val_acc: 0.9562
Epoch 196/200
13389/13389 [==============================] - 0s - loss: 0.0861 - acc: 0.9597 - val_loss: 0.0832 - val_acc: 0.9597
Epoch 197/200
13389/13389 [==============================] - 0s - loss: 0.0929 - acc: 0.9541 - val_loss:

In [23]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.9238817285822593


In [24]:
from sklearn.metrics import classification_report,confusion_matrix
Cm = confusion_matrix(y_eval,pred)
C = np.sum(Cm)
Cm = Cm/C
print('Confusion Matrix:')
print(np.array_str(Cm, precision=4, suppress_small=True))

Confusion Matrix:
[[ 0.0942  0.    ]
 [ 0.0761  0.8297]]


In [ ]:
valAcc = history_cb.val_acc
valLoss = history_cb.val_loss
epoch_it = np.arange(1,201)
plt.subplot(121)
plt.plot(epoch_it, valAcc)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Validation accuracy')
plt.tight_layout()
plt.subplot(122)
plt.plot(epoch_it, valLoss)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Validation loss')
plt.tight_layout()
plt.show()

In [ ]:
Ls = history_cb.losses
Acc = history_cb.accs
ntr = np.shape(x_train)[0]
epochNum = []
for i in range(24200):
    epochNum.append(i*100/ntr)

matplotlib.pyplot.semilogy(epochNum, Ls)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.tight_layout()
plt.show()

In [ ]:
matplotlib.pyplot.semilogy(epochNum, Acc)
# plt.plot(epochNum, Acc)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Training Accuracy')
plt.tight_layout()
# plt.axis([0.3,3,0.96,1])
plt.show()



In [ ]:
# # Save & Load result
# import os
# import tensorflow as tf
# saver = tf.train.Saver()

In [ ]:
# save_dir = 'checkpoints/' 

In [ ]:
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

In [ ]:
# save_path = os.path.join(save_dir, 'best_validation')

In [ ]:
# saver.save(sess=session, save_path=save_path)

In [ ]:
# saver.restore(sess=session, save_path=save_path)

In [ ]:
# session.run(tf.global_variables_initializer())

In [ ]:
# print_test_accuracy(show_example_errors=True)